## Building Predective Models

In [ ]:
import numpy as np
import pandas as pd
import os

### Import file

In [ ]:
train_test_path=os.path.join(os.path.pardir,'data','processed')
train_path=os.path.join(train_test_path,'train.csv')
test_path=os.path.join(train_test_path,'test.csv')

In [ ]:
train_df=pd.read_csv(train_path,index_col='PassengerId')
test_df=pd.read_csv(test_path,index_col='PassengerId')

In [ ]:
train_df.info()

In [103]:
train_df.columns

Index(['Survived', 'Age', 'Fare', 'FamilySize', 'IsMother', 'Ismale', 'Deck_A',
       'Deck_B', 'Deck_C', 'Deck_D', 'Deck_E', 'Deck_F', 'Deck_G', 'Deck_Z',
       'Pclass_1', 'Pclass_2', 'Pclass_3', 'Title_Lady', 'Title_Master',
       'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Officer', 'Title_Sir',
       'Fare_Bin_very_low', 'Fare_Bin_low', 'Fare_Bin_high',
       'Fare_Bin_very_high', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
       'AgeState_Adult', 'AgeState_Child'],
      dtype='object')

### Data Prepration

In [ ]:
X=train_df.loc[:,'Age':].as_matrix().astype('float')
y=train_df['Survived'].ravel()

In [ ]:
print(X.shape)

In [ ]:
print(y.shape)

In [ ]:
#train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
#Average Survival in train and test
print('mean survival in train {0:.3f}'.format(np.mean(y_train)))
print('mean survival in test {0:.3f}'.format(np.mean(y_test)))

#### check scikit learn version

In [ ]:
import sklearn

In [ ]:
sklearn.__version__

In [ ]:
#make sure v is greater than 0.19.0
#!conda update -y scikit-learn

### Baseline Model

In [ ]:
#import function
from sklearn.dummy import DummyClassifier

In [ ]:
#create model
model_dummy=DummyClassifier(strategy='most_frequent', random_state=0)

In [ ]:
#train_model
model_dummy.fit(X_train,y_train)

In [ ]:
print('score of baseline model: {0:.3f}'.format(model_dummy.score(X_test,y_test)))

In [ ]:
#Performance metrix
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score


In [ ]:
#accuracy score
print('Accuracy of baseline model is: {0:.3f}'.format(accuracy_score(y_test,model_dummy.predict(X_test))))

In [ ]:
#confusion matrix
print('confusion of baseline model is: {0}'.format(confusion_matrix(y_test,model_dummy.predict(X_test))))

In [ ]:
#precision and recall score
print('precision of baseline model is: {0}'.format(precision_score(y_test,model_dummy.predict(X_test))))
print('recall of baseline model is: {0}'.format(recall_score(y_test,model_dummy.predict(X_test))))

### First kaggle submission

In [ ]:
#convert to matrix
test_X=test_df.as_matrix().astype('float')

In [ ]:
#get predections
predictions=model_dummy.predict(test_X)

In [ ]:
df_submission=pd.DataFrame({'PassengerId':test_df.index,'Survived':predictions})

In [ ]:
df_submission.head()

In [ ]:
submission_data_path=os.path.join(os.path.pardir,'data','external')
submission_file_path=os.path.join(submission_data_path,'1_dummy.csv')

In [ ]:
df_submission.to_csv(submission_file_path,index=False)

In [ ]:
def get_submission_file(model,filename):
    test_X=test_df.as_matrix().astype('float')
    predictions=model.predict(test_X)
    df_submission=pd.DataFrame({'PassengerId':test_df.index,'Survived':predictions})
    submission_data_path=os.path.join(os.path.pardir,'data','external')
    submission_file_path=os.path.join(submission_data_path,filename)
    df_submission.to_csv(submission_file_path,index=False)

### logestic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model_lr_1=LogisticRegression(random_state=0)

In [ ]:
model_lr_1.fit(X_train,y_train)

In [ ]:
print('Score of LR: {0:.3f}'.format(model_lr_1.score(X_test,y_test)))

In [ ]:
model_lr_1.coef_

## Second Submission

In [ ]:
get_submission_file(model_lr_1,'2_lr.csv')

### Hyperparameter Optimization

In [ ]:
model_lr=LogisticRegression(random_state=0)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters={'C':[1.0,10.0,50.0,100.0,1000],'penalty':['l1','l2']}
clf=GridSearchCV(model_lr,param_grid=parameters,cv=3)

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
clf.best_params_

In [ ]:
print('Best Score is: {0:.2f}'.format(clf.best_score_))

### Making third submission

In [ ]:
get_submission_file(clf,'3_lr.csv')

## Feature Normalization and Standardisation

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [ ]:
#Feature normalization
scaler=MinMaxScaler()
X_train_scaled=scaler.fit_transform(X_train)

In [ ]:
X_train_scaled[:,0].min()

In [ ]:
X_train_scaled[:,0].max()

In [ ]:
#normalise test data
X_test_scaled=scaler.fit_transform(X_test)

## Feature Standarisation

In [ ]:
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [ ]:
print('Score for  file is: {0:.3f}'.format(clf.score(X_test_scaled,y_test)))

### Model Persistence

In [ ]:
#import pickle library
import pickle

In [ ]:
#create the file paths
model_file_path=os.path.join(os.path.pardir,'models','lr_model.pkl')
scaler_file_path=os.path.join(os.path.pardir,'models','lr_scalar.pkl')

In [ ]:
#open the files to write
model_file_pickle=open(model_file_path,'wb')
scaler_file_pickle=open(scaler_file_path,'wb')

In [ ]:
#write in file
pickle.dump(clf,model_file_pickle)
pickle.dump(scaler,scaler_file_pickle)

In [ ]:
#close the file
model_file_pikle.close()
scaler_file_pikle.close()

## load the persisted file to check if file is working or not

In [ ]:
#open files in read mode
model_file_pickle=open(model_file_path,'rb')
scaler_file_pickle=open(scaler_file_path,'rb')

In [ ]:
#load files
clf_loaded=pickle.load(model_file_pickle)
scaler_loaded=pickle.load(scaler_file_pickle)
#close files
model_file_pickle.close()
scaler_file_pickle.close()

In [ ]:
clf_loaded

In [ ]:
scaler_loaded

In [ ]:
#test on test data
X_test_scaled=scaler_loaded.transform(X_test)

In [ ]:
print('Score for persisted file is: {0:.3f}'.format(clf_loaded.score(X_test_scaled,y_test)))